In [14]:
import sys
sys.path.append('..')
from BDDData import *
import numpy as np
import torch
import torch.utils.data as data

import networkx as nx
from graph.product_graph import *
from utils import *

import torch
import torch.nn as nn
import torch.nn.functional as F


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
#Load dataframes
bdd_data = BDD_dataset("../raw_data/BDDdata/")
#Add column with the timestep
bdd_data.add_timestep_id()
#Add flags for chaotic values
bdd_data.tag_chaotic(replace=True)
#Compute the mod for the nazelle and wind angles
bdd_data.angle_mod()
#Interpolate the missing values
bdd_data.interpolate_power()
#Values smaller than 0 are set to 0
bdd_data.cap_power_to_zero()
#Normalize Patv feature to [0,1]
bdd_data.normalize_power(min=0, max=1, method= "MinMaxScaler")
#Convert df to matrix form, where only Patv is included. Then split into train, validation and test
#The matrix contains the subset of the time series for ALL nodes, so an (TxN matrix)
train, val, test = bdd_data.split_df()

In [6]:
# set seed
np.random.seed(42)
# consider a p percentage of the data
p = 1.00
train_mask = np.random.choice(train.shape[1], int(train.shape[1] * p), replace=False)
val_mask = np.random.choice(val.shape[1], int(val.shape[1] * p), replace=False)
test_mask = np.random.choice(test.shape[1], int(test.shape[1] * p), replace=False)

train = train[:, train_mask]
val = val[:, val_mask]
test = test[:, test_mask]

In [7]:
class CustomBDD_Dataset(data.Dataset):
    def __init__(self, dataset, observation_window=12, forecast_window=12, starting_turbine = 0,  ending_turbine=133):
        self.observation_window = observation_window
        self.forecast_window = forecast_window
        length = eval(f'len({dataset}[0])')#Retrieves length of dataset
        bdd_data.get_observation_forecasting_window(time_series_len=length, observation_steps=self.observation_window, forecast_steps=self.forecast_window)#Generates obs window
        self.window_of_interest =  bdd_data.sliding_indices[str(self.observation_window)+","+str(self.forecast_window)]#Retrieves windows
        self.starting_turbine = starting_turbine
        self.ending_turbine = ending_turbine  
        self.dataset = dataset

    def __len__(self):
        return len(self.window_of_interest)

    def __getitem__(self, idx):
        window = self.window_of_interest[idx]
        if self.dataset == "train":
            features = train[self.starting_turbine:self.ending_turbine+1,window[0]:window[1]]
            labels = train[self.starting_turbine:self.ending_turbine+1,window[1]:window[2]]
        elif self.dataset == "val":
            features = val[self.starting_turbine:self.ending_turbine+1,window[0]:window[1]]
            labels = val[self.starting_turbine:self.ending_turbine+1,window[1]:window[2]]
        elif self.dataset == "test":
            features = test[self.starting_turbine:self.ending_turbine+1,window[0]:window[1]]
            labels = test[self.starting_turbine:self.ending_turbine+1,window[1]:window[2]]
        else:
            raise NotImplementedError
        return torch.from_numpy(features).float(), torch.from_numpy(labels).float()
    
obs_window = 12
forecast_window = 12
batch_size = 100
num_nodes = 134

train_dataset = CustomBDD_Dataset("train",observation_window=obs_window,forecast_window=forecast_window)
train_loader = data.DataLoader(train_dataset, shuffle=True, batch_size = batch_size)
val_dataset = CustomBDD_Dataset("val",observation_window=obs_window,forecast_window=forecast_window)
val_loader = data.DataLoader(val_dataset, shuffle=True, batch_size = batch_size)
test_dataset = CustomBDD_Dataset("test",observation_window=obs_window,forecast_window=forecast_window)
test_loader = data.DataLoader(test_dataset, shuffle=True, batch_size = batch_size)

In [15]:
x,y = next(iter(train_loader))
print(f"{x.shape=}\n{y.shape=}")

x.shape=torch.Size([100, 134, 12])
y.shape=torch.Size([100, 134, 12])


In [16]:
G = nx.read_gml('../graph/data/spatial_graph_2000.gml')
adj_mat = nx.adjacency_matrix(G)
adj_mat = nx.to_numpy_array(G)

In [17]:
S = normalize_adjacency(torch.tensor(adj_mat)).float().to(device)

In [18]:
print(adj_mat.shape)

(134, 134)


In [19]:
class GCNNLayer(nn.Module):
    def __init__(self, in_features, out_features, matrix_powers, order):
        super(GCNNLayer, self).__init__()
        self.matrix_powers = matrix_powers
        self.order = order
        self.weights = nn.Parameter(torch.FloatTensor(in_features, out_features, order+1))
        # use Xavier initialization to match variance of input with output
        nn.init.xavier_uniform_(self.weights)

    def forward(self, features):
        batch_size = features.size(0)
        output_dim = self.weights.size(1)
        device = features.device

        out = torch.zeros((batch_size, features.size(1), output_dim), device=device)
        for k in range(self.order+1):
            weighted = torch.bmm(features, self.weights[:, :, k].unsqueeze(0).repeat(batch_size, 1, 1))
            shifted = torch.bmm(self.matrix_powers[k].unsqueeze(0).repeat(batch_size, 1, 1).to(device), weighted)
            out += shifted
        return out

# Inputs must be sized [num_nodes, obs_size] and outputs will be [num_nodes, pred_size]
class GCNN(nn.Module):
    def __init__(self, hid_sizes, shift, order):
        super(GCNN, self).__init__()
        self.layers = nn.ModuleList()
        # compute matrix shift
        matrix_powers = [torch.matrix_power(shift, k).float() for k in range(order+1)]
        # input layer of size obs_size
        self.layers.append(GCNNLayer(obs_window, hid_sizes[0], matrix_powers, order))
        # num_hid hidden layers of size hid_size
        for i in range(len(hid_sizes) - 1):
            self.layers.append(GCNNLayer(hid_sizes[i], hid_sizes[i + 1], matrix_powers, order))

    def forward(self, features):
        temp = features
        for layer in self.layers[:-1]:
            # use relu activation function
            temp = F.relu(layer(temp))
        return self.layers[-1](temp)

In [20]:
import time

def train_epoch(model, loader, optimizer, device='cpu'):
    model.train()
    total_loss = 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = torch.nn.functional.mse_loss(outputs, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate_epoch(model, loader, device='cpu'):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = torch.nn.functional.mse_loss(outputs, y)
            total_loss += loss.item()
    return total_loss / len(loader)

def train_model(model, train_loader, val_loader, file_name, num_epochs=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)

    start_time = time.time()
    train_losses = []
    val_losses = []
    pointers = []
    best_loss = None

    for epoch in range(1, num_epochs + 1):
        train_loss = train_epoch(model, train_loader, optimizer, device=device)
        val_loss = evaluate_epoch(model, val_loader, device=device)

        # If model is better, export new model pointer.
        if best_loss is None or val_loss < best_loss:
            best_loss = val_loss
            pointers.append(epoch)
            torch.save(model.state_dict(), f'{file_name}_v{epoch}.pth')

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        if epoch % 1 == 0:
            print(f"epoch: {epoch}\ttraining loss: {train_loss:.4f}\tvalidation loss: {val_loss:.4f}")

    elapsed_time = time.time() - start_time
    if device.type == 'cuda':
        torch.cuda.synchronize()
    print(f'Model training took {elapsed_time:.3f} seconds')

    return train_losses, val_losses, pointers, elapsed_time

In [22]:
architectures = [[16, 16, 16, forecast_window], [8, 8, 8, forecast_window]] # [16, 8, forecast_window], [16, 8, 4, forecast_window]
order = 1
num_epochs = 10

for architecture in architectures:
        model = GCNN(architecture, S, order).to(device)
        architecture_string = '[' + ','.join(map(str, architecture)) + ']'
        file_name = f'../results/gcnn_e={num_epochs}_d={p}_k={order}{architecture_string}'

        pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Created GCNN model with {pytorch_total_params} parameters:")
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(name, param.data.size())

        # Train the model
        train_losses, val_losses, pointers, elapsed_time = train_model(model, train_loader, val_loader, file_name, num_epochs=num_epochs)

        # Run the best model on the test set, export accuracy
        best_loc = f'{file_name}_v{pointers[-1]}.pth'
        model.load_state_dict(torch.load(best_loc))
        test_loss = evaluate_epoch(model, test_loader, device=device)
        print(f"Final test loss: {test_loss}\n")

        # Export test accuracy
        file_path = f'{file_name}.txt'
        with open(file_path, 'w') as f:
            f.write(f"{str(test_loss)}, {elapsed_time:.3f}\n")

Created GCNN model with 1792 parameters:
layers.0.weights torch.Size([12, 16, 2])
layers.1.weights torch.Size([16, 16, 2])
layers.2.weights torch.Size([16, 16, 2])
layers.3.weights torch.Size([16, 12, 2])
epoch: 1	training loss: 0.0884	validation loss: 0.0616
epoch: 2	training loss: 0.0763	validation loss: 0.0639


KeyboardInterrupt: 

In [24]:
#Load dataframes
bdd_data = BDD_dataset("../raw_data/BDDdata/")
#Add column with the timestep
bdd_data.add_timestep_id()
#Add flags for chaotic values
bdd_data.tag_chaotic(replace=True)
#Compute the mod for the nazelle and wind angles
bdd_data.angle_mod()
#Interpolate the missing values
bdd_data.interpolate_power()
#Values smaller than 0 are set to 0
bdd_data.cap_power_to_zero()
#Normalize Patv feature to [0,1]
bdd_data.normalize_power(min=0, max=1, method= "MinMaxScaler")
#Convert df to matrix form, where only Patv is included. Then split into train, validation and test
#The matrix contains the subset of the time series for ALL nodes, so an (TxN matrix)
_, _, test = bdd_data.split_df()

test_dataset_nan = CustomBDD_Dataset("test",observation_window=obs_window,forecast_window=forecast_window)
test_loader_nan = data.DataLoader(test_dataset, shuffle=False, batch_size = batch_size)

In [ ]:
#Load dataframes
bdd_data = BDD_dataset("../raw_data/BDDdata/")
#Add column with the timestep
bdd_data.add_timestep_id()
#Add flags for chaotic values
bdd_data.tag_chaotic(replace=True)
#Compute the mod for the nazelle and wind angles
bdd_data.angle_mod()
#Interpolate the missing values
bdd_data.interpolate_power()
#Values smaller than 0 are set to 0
bdd_data.cap_power_to_zero()
#Normalize Patv feature to [0,1]
bdd_data.normalize_power(min=0, max=1, method= "MinMaxScaler")
#Convert df to matrix form, where only Patv is included. Then split into train, validation and test
#The matrix contains the subset of the time series for ALL nodes, so an (TxN matrix)
_, _, test = bdd_data.split_df()

test_dataset_interpolated = CustomBDD_Dataset("test",observation_window=obs_window,forecast_window=forecast_window)
test_loader_interpolated = data.DataLoader(test_dataset, shuffle=False, batch_size = batch_size)

In [ ]:
# Helper functions
class SDELoss(nn.Module):
    def __init__(self):
        super(SDELoss, self).__init__()

    def forward(self, input, target):
        error = input - target
        mean_error = torch.mean(error)
        return torch.sqrt(torch.mean((error - mean_error) ** 2))
    
class MaskedLoss(nn.Module):
    def __init__(self, criterion):
        super(MaskedLoss, self).__init__()
        self.criterion = criterion

    def forward(self, prediction, target):
        # Create a mask that is 1 for non-NaN entries and 0 for NaN entries
        isnan = torch.isnan(target)
        mask = ~isnan
        # Apply the mask to only keep non-NaN elements
        out = prediction[mask]
        tar = target[mask]
        if torch.any(torch.isnan(out)):
            print("X")
        # Calculate MSE Loss on non-NaN elements
        return self.criterion(out, tar)


def evaluate_epoch(model, interpolated_loader, nan_loader, criterion, device='cpu'):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        nan_iter = iter(nan_loader)
        for x, _ in interpolated_loader:
            _, y = next(nan_iter)
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()
    return total_loss / len(interpolated_loader)

# GCNN
architecture = [8,8,12]
model = GCNN(architecture, S, order).to(device)
path = "results/gcnn_d=1.00/gcnn_e=10_d=1.0_k=1[8,8,12]_v2.pth"
model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))

print(np.sqrt(evaluate_epoch(model, test_loader_interpolated, test_loader_nan, MaskedLoss(nn.MSELoss()), device=device)))
print(evaluate_epoch(model, test_loader_interpolated, test_loader_nan, MaskedLoss(nn.L1Loss()), device=device))
print(evaluate_epoch(model, test_loader_interpolated, test_loader_nan, MaskedLoss(SDELoss()), device=device))

0.13360085495309934
0.083043375732525
0.10187376062903139
